Same setup as before

In [1]:
import os
os.chdir("../..")
import django_init


In [2]:
# For better printing

from rich import print
from rich.console import Console
from rich.markdown import Markdown

console = Console()

In [3]:
from utils.perf_display import format_duration, perf_counter

In [4]:
from books.models import Library, Review, Book
from django.db.models import Count

medium = Library.objects.order_by("-id").first()
medium_id = medium.id

# Get the library with the most books
alexandria_id, book_count = (
    Book.objects.values("library_id")
    .annotate(book_count=Count("id"))
    .order_by("-book_count")
    .values_list("library_id", "book_count")
    .first()
)

alexandria = Library.objects.get(id=alexandria_id)

## 2nd exercice
Let's continue with another usual example. 
We want to implement a standard endpoint returning a page of filtered and ordered reviews.
A minimal implementation is available in [this file]("../../../../books/views/review/simple.py). 

In [5]:
# Let's try the existing endpoint
from rest_framework.test import APIClient
client = APIClient()
%time result = client.get(f"/reviews/{alexandria_id}/simple")


CPU times: user 53.9 ms, sys: 7.69 ms, total: 61.6 ms
Wall time: 82.4 ms


/home/laurent/.local/share/virtualenvs/django-perfs-playground-tFy-cSse/lib/python3.10/site-packages/rest_framework/pagination.py:200: UnorderedObjectListWarning: Pagination may yield inconsistent results with an unordered object_list: <class 'books.models.review.Review'> QuerySet.
  paginator = self.django_paginator_class(queryset, page_size)


In [6]:
result = client.get(f"/reviews/{alexandria_id}/ordered?ordering=-wrote_at")

In [7]:
result = client.get(f"/reviews/{alexandria_id}/ordered?ordering=id")

In [8]:
from books.tests.benchmarks import benchmark_list_reviews

benchmark_list_reviews(["simple-list-reviews", "ordered-list-reviews"], library_ids=[medium_id, alexandria_id])

                                          Endpoint benchmark                                          
┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━┓
┃ endpoint URL           ┃ Query params              ┃ Library ┃ Mean duration ┃ Max duration ┃ Runs ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━┩
│ /reviews/10015/simple  │                           │ 10015   │ 0.018 s       │ 0.018 s      │ 1    │
├────────────────────────┼───────────────────────────┼─────────┼───────────────┼──────────────┼──────┤
│ /reviews/6/simple      │                           │ 6       │ 0.016 s       │ 0.016 s      │ 1    │
├────────────────────────┼───────────────────────────┼─────────┼───────────────┼──────────────┼──────┤
│ /reviews/10015/ordered │ ordering=id               │ 10015   │ 0.497 s       │ 0.497 s      │ 1    │
├────────────────────────┼───────────────────────────┼─────────┼───────────────┼──────────────┼──────┤
│ /reviews/6/ordered     │ ordering=id               │ 6       │ 0.022 s       │ 0.022 s      │ 1    │
├────────────────────────┼───────────────────────────┼─────────┼───────────────┼──────────────┼──────┤
│ /reviews/10015/ordered │ ordering=wrote_at         │ 10015   │ 0.199 s       │ 0.199 s      │ 1    │
├────────────────────────┼───────────────────────────┼─────────┼───────────────┼──────────────┼──────┤
│ /reviews/6/ordered     │ ordering=wrote_at         │ 6       │ 8.254 s       │ 8.254 s      │ 1    │
├────────────────────────┼───────────────────────────┼─────────┼───────────────┼──────────────┼──────┤
│ /reviews/10015/ordered │ ordering=rating           │ 10015   │ 0.193 s       │ 0.193 s      │ 1    │
├────────────────────────┼───────────────────────────┼─────────┼───────────────┼──────────────┼──────┤
│ /reviews/6/ordered     │ ordering=rating           │ 6       │ 4.018 s       │ 4.018 s      │ 1    │
├────────────────────────┼───────────────────────────┼─────────┼───────────────┼──────────────┼──────┤
│ /reviews/10015/ordered │ ordering=-id              │ 10015   │ 0.125 s       │ 0.125 s      │ 1    │
├────────────────────────┼───────────────────────────┼─────────┼───────────────┼──────────────┼──────┤
│ /reviews/6/ordered     │ ordering=-id              │ 6       │ 50.224 s      │ 50.224 s     │ 1    │
├────────────────────────┼───────────────────────────┼─────────┼───────────────┼──────────────┼──────┤
│ /reviews/10015/ordered │ ordering=-wrote_at        │ 10015   │ 0.391 s       │ 0.391 s      │ 1    │
├────────────────────────┼───────────────────────────┼─────────┼───────────────┼──────────────┼──────┤
│ /reviews/6/ordered     │ ordering=-wrote_at        │ 6       │ 6.472 s       │ 6.472 s      │ 1    │
├────────────────────────┼───────────────────────────┼─────────┼───────────────┼──────────────┼──────┤
│ /reviews/10015/ordered │ ordering=-rating          │ 10015   │ 0.120 s       │ 0.120 s      │ 1    │
├────────────────────────┼───────────────────────────┼─────────┼───────────────┼──────────────┼──────┤
│ /reviews/6/ordered     │ ordering=-rating          │ 6       │ 2.687 s       │ 2.687 s      │ 1    │
├────────────────────────┼───────────────────────────┼─────────┼───────────────┼──────────────┼──────┤
│ /reviews/10015/ordered │ ordering=-wrote_at,rating │ 10015   │ 0.152 s       │ 0.152 s      │ 1    │
├────────────────────────┼───────────────────────────┼─────────┼───────────────┼──────────────┼──────┤
│ /reviews/6/ordered     │ ordering=-wrote_at,rating │ 6       │ 2.659 s       │ 2.659 s      │ 1    │
└────────────────────────┴───────────────────────────┴─────────┴───────────────┴──────────────┴──────┘

In [9]:
def explain_qs(library_id, ordering):
    with perf_counter(message=f"ordering: {ordering}", time_sql=True, print_sql=True):
        result =  (
            Review.objects.filter(library_id=library_id)
            .values("id")
            .order_by(ordering)[:20]
            .explain(analyze=True)
        )
    print(result)
    console.rule()


explain_qs(medium_id, "id")
explain_qs(medium_id, "-id")
explain_qs(alexandria_id, "id")
explain_qs(alexandria_id, "-id")

ordering: id: 0.08s.
SQL: 0.08s.
       (duration: 0.082)   EXPLAIN (ANALYZE true) SELECT "books_review"."id" FROM "books_review" WHERE 
"books_review"."library_id" = 10015 ORDER BY "books_review"."id" ASC LIMIT 20

Limit  (cost=4187.02..4187.07 rows=20 width=8) (actual time=77.768..77.772 rows=20 loops=1)
  ->  Sort  (cost=4187.02..4193.02 rows=2400 width=8) (actual time=77.766..77.768 rows=20 loops=1)
        Sort Key: id
        Sort Method: top-N heapsort  Memory: 25kB
        ->  Index Scan using books_review_library_id_e9b4afba on books_review  (cost=0.44..4123.15 rows=2400 
width=8) (actual time=0.051..51.778 rows=200789 loops=1)
              Index Cond: (library_id = 10015)
Planning Time: 0.116 ms
Execution Time: 77.823 ms

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

ordering: -id: 0.12s.
SQL: 0.12s.
       (duration: 0.115)   EXPLAIN (ANALYZE true) SELECT "books_review"."id" FROM "books_review" WHERE 
"books_review"."library_id" = 10015 ORDER BY "books_review"."id" DESC LIMIT 20

Limit  (cost=4187.02..4187.07 rows=20 width=8) (actual time=101.355..101.361 rows=20 loops=1)
  ->  Sort  (cost=4187.02..4193.02 rows=2400 width=8) (actual time=101.352..101.355 rows=20 loops=1)
        Sort Key: id DESC
        Sort Method: top-N heapsort  Memory: 26kB
        ->  Index Scan using books_review_library_id_e9b4afba on books_review  (cost=0.44..4123.15 rows=2400 
width=8) (actual time=0.050..67.756 rows=200789 loops=1)
              Index Cond: (library_id = 10015)
Planning Time: 0.138 ms
Execution Time: 101.636 ms

...

Limit  (cost=0.44..23.51 rows=20 width=8) (actual time=3.188..3.327 rows=20 loops=1)
  ->  Index Scan using books_review_pkey on books_review  (cost=0.44..2358007.46 rows=2044265 width=8) (actual 
time=3.186..3.322 rows=20 loops=1)
        Filter: (library_id = 6)
        Rows Removed by Filter: 3276
Planning Time: 0.129 ms
Execution Time: 3.366 ms

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

ordering: -id: 9.10s.
SQL: 9.10s.
       (duration: 9.096)   EXPLAIN (ANALYZE true) SELECT "books_review"."id" FROM "books_review" WHERE 
"books_review"."library_id" = 6 ORDER BY "books_review"."id" DESC LIMIT 20

Limit  (cost=0.44..23.51 rows=20 width=8) (actual time=9095.202..9095.224 rows=20 loops=1)
  ->  Index Scan Backward using books_review_pkey on books_review  (cost=0.44..2358007.46 rows=2044265 width=8) 
(actual time=9095.200..9095.220 rows=20 loops=1)
        Filter: (library_id = 6)
        Rows Removed by Filter: 20010072
Planning Time: 0.097 ms
Execution Time: 9095.280 ms

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [10]:
explain_qs(alexandria_id, "wrote_at")

ordering: wrote_at: 3.19s.
SQL: 3.19s.
       (duration: 3.188)   EXPLAIN (ANALYZE true) SELECT "books_review"."id" FROM "books_review" WHERE 
"books_review"."library_id" = 6 ORDER BY "books_review"."wrote_at" ASC LIMIT 20

Limit  (cost=1449258.88..1449261.21 rows=20 width=16) (actual time=3165.938..3185.807 rows=20 loops=1)
  ->  Gather Merge  (cost=1449258.88..1648020.72 rows=1703554 width=16) (actual time=3109.519..3129.384 rows=20 
loops=1)
        Workers Planned: 2
        Workers Launched: 2
        ->  Sort  (cost=1448258.85..1450388.30 rows=851777 width=16) (actual time=3084.792..3084.795 rows=17 
loops=3)
              Sort Key: wrote_at
              Sort Method: top-N heapsort  Memory: 26kB
              Worker 0:  Sort Method: top-N heapsort  Memory: 26kB
              Worker 1:  Sort Method: top-N heapsort  Memory: 26kB
              ->  Parallel Seq Scan on books_review  (cost=0.00..1425593.38 rows=851777 width=16) (actual 
time=80.984..2958.591 rows=668891 loops=3)
                    Filter: (library_id = 6)
                    Rows Removed by Filter: 12666173
Planning Time: 0.119 ms
JIT:
  Functions: 13
  Options: Inlining true, Optimization true, Expressions true, Deforming true
  Timing: Generation 2.719 ms, Inlining 165.461 ms, Optimization 83.760 ms, Emission 49.940 ms, Total 301.881 ms
Execution Time: 3186.674 ms

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

None

In [ ]:
from utils.sql import indexes_size
indexes_size()